### INFO 624 Final Project:
   #### IR website with elastic search and skiing website 
   - https://www.myweather2.com/developer/apis.aspx?uref=f8add9ec-fe95-4001-b822-df8680e49dce![image.png](attachment:image.png)
   - https://www.myweather2.com/ski-resorts.aspx
   ------
   - possible queries:
   

In [6]:
from elasticsearch import Elasticsearch, helpers; from bs4 import BeautifulSoup
import json, requests, csv, sys, os, re, collections, webbrowser
from collections import OrderedDict

In [80]:
resort_towns = ['Vail,CO','Beaver Creek,CO','Breckenridge,CO','Keystone,CO','Crested Butte,CO',
 'Park City,UT','Heavenly, CA/NV','Northstar,CA','Kirkwood,CA','Stevens Pass,WA',
 'Stowe, VT','Okemo, VT','Mount Snow, VT','Mount Sunapee, NH','Wildcat Mountain, NH',
 'Attitash Mountain, NH','Crotched Mountain, NH','Hunter Mountain, NY',
 'Liberty Mountain Resort, PA','Roundtop Mountain Resort, PA','White tail Resort, PA',
 'Jack frost, PA', 'Big Boulder, PA','Alpine Valley, OH','Mad River Mountain, OH',
 'Hidden Valley, MO','Snow Creek, MO', 'Paoli Peaks, IN','Wilmot, WI','Afton Alps, MN',
 'Mt Brighton, MI','Telluride, CO','Sun Valley, ID','Snowbasin, UT', 'Winter Park, CO',
 'Copper Mountain, CO', 'Steamboat, CO', 'Eldora Mountain, CO', 'Palisades Tahoe, CA',
 'Mammoth Mountain, CA', 'June Mountain, CA', 'Big Bear Mountain Resort, CA', 'Stratton, VT',
 'Snowshoe, WV', 'Sugarbush, VT','Solitude, UT']

reduced_resort_towns = ['Vail,CO','Beaver Creek,CO','Breckenridge,CO','Keystone,CO','Crested Butte,CO',
 'Park City,UT','Heavenly, CA/NV','Northstar,CA','Kirkwood,CA','Stevens Pass,WA',
 'Stowe, VT','Okemo, VT','Mount Snow, VT','Mount Sunapee, NH','Wildcat Mountain, NH',
 'Attitash Mountain, NH','Crotched Mountain, NH','Hunter Mountain, NY']

In [105]:
def get_resorts(resort_list):
    base_and_key = 'http://api.worldweatheronline.com/premium/v1/ski.ashx?key=e645267bfab34ccd968203330221103&q='
    formatt = '&format=json'; responses=[]
    
    for i in resort_list:
        ## check if requesst worked or not, skip if it didn't
        current_request = base_and_key+i+formatt
        response = requests.get(current_request)
        json_string = json.loads(response.text)
        
        if 'error' in list(json_string.get('data').keys()):
            continue # if location isn't found, skip iteration
            
        #split request and weather if location is found
        requestt = json_string.get('data').get('request')[0]
        weatherr = json_string.get('data').get('weather')[0]
        
        ## request
        query = requestt.get('query'); query_type = requestt.get('type')
        ## split weather up to get desired keys
        chance_of_snow = weatherr.get('chanceofsnow')
        totalSnowFall_cm = weatherr.get('totalSnowfall_cm')
        hourlyConditions_mid = weatherr.get('hourly')[0].get('mid')[0]
        temp_F = hourlyConditions_mid.get('tempF')
        weather_desc = hourlyConditions_mid.get('weatherDesc')[0].get('value')
        windSpeed_mph = hourlyConditions_mid.get('windspeedMiles')
        
        
        ## create new dict with reduced keys and values
        keys_of_interest = ['query','query_type','weather_desc','chance_of_snow',
          'totalsnowfall_cm','temp_f','windspeed_mph'];values_of_interest = []
        values_of_interest.extend([query,query_type,weather_desc,chance_of_snow,
                                    totalSnowFall_cm,temp_F,windSpeed_mph])
        reduced_results= dict(zip(keys_of_interest, values_of_interest))
        responses.append(reduced_results)
        
    return responses

def for_testing(resort_list):
    base_and_key = 'http://api.worldweatheronline.com/premium/v1/ski.ashx?key=e645267bfab34ccd968203330221103&q='
    formatt = '&format=json'; responses=[]
    
    for i in resort_list:
        #get request of current town and get get json/dict
        current_request = base_and_key+i+formatt
        response = requests.get(current_request)
        json_string = json.loads(response.text)
    responses.append(json_string)
    return responses

In [108]:
#state #data #snowfall #id
resort_responses = get_resorts(resort_towns)
print(len(resort_responses)); resort_responses[0:3]

41


[{'query': 'Vail,CO',
  'query_type': 'City',
  'weather_desc': 'Heavy snow',
  'chance_of_snow': '47',
  'totalsnowfall_cm': '8.3',
  'temp_f': '18',
  'windspeed_mph': '4'},
 {'query': 'Beaver Creek,CO',
  'query_type': 'City',
  'weather_desc': 'Heavy snow',
  'chance_of_snow': '47',
  'totalsnowfall_cm': '7.9',
  'temp_f': '19',
  'windspeed_mph': '4'},
 {'query': 'Breckenridge,CO',
  'query_type': 'City',
  'weather_desc': 'Patchy heavy snow',
  'chance_of_snow': '61',
  'totalsnowfall_cm': '4.2',
  'temp_f': '13',
  'windspeed_mph': '11'}]

In [ ]:
####

In [56]:
testingg = for_testing(reduced_resort_towns)
testt = testingg[0] # A single resort 
key_list = list(testt.get('data').keys()); print('Initial Keys: ', key_list) 

## split request up 
query = testt.get('data').get('request')[0].get('query') 
query_type = testt.get('data').get('request')[0].get('type')
print('Query: ',query, '   Query type: ', query_type)

## display weather keys
weather_keys = list(testt.get('data').get('weather')[0].keys())
print('\nKeys within Weather: \n', weather_keys)

## split weather up to get desired keys
chance_of_snow = testt.get('data').get('weather')[0].get('chanceofsnow')
totalSnowFall_cm = testt.get('data').get('weather')[0].get('totalSnowfall_cm')
hourlyConditions_mid = testt.get('data').get('weather')[0].get('hourly')[0].get('mid')[0]
print('\nHourly Conditions mid keys: \n', list(hourlyConditions_mid.keys()))
temp_F = hourlyConditions_mid.get('tempF')
weather_desc = hourlyConditions_mid.get('weatherDesc')[0].get('value')
windSpeed_mph = hourlyConditions_mid.get('windspeedMiles')

Initial Keys:  ['request', 'weather']
Query:  Hunter Mountain, NY    Query type:  City

Keys within Weather: 
 ['astronomy', 'bottom', 'chanceofsnow', 'date', 'hourly', 'mid', 'top', 'totalSnowfall_cm']

Hourly Conditions mid keys: 
 ['tempC', 'tempF', 'weatherCode', 'weatherDesc', 'weatherIconUrl', 'winddir16Point', 'winddirDegree', 'windspeedKmph', 'windspeedMiles']


In [57]:
## create new dict with reduced keys and values
keys_of_interest = ['query','query_type','weather_desc','chance_of_snow',
 'totalsnowfall_cm','temp_f','windspeed_mph'];values_of_interest = []

values_of_interest.extend([query,query_type,weather_desc,chance_of_snow,
                           totalSnowFall_cm,temp_F,windSpeed_mph])

reduced_results= dict(zip(keys_of_interest, values_of_interest))
reduced_results

{'query': 'Hunter Mountain, NY',
 'query_type': 'City',
 'weather_desc': 'Clear',
 'chance_of_snow': '0',
 'totalsnowfall_cm': '0.9',
 'temp_f': '42',
 'windspeed_mph': '11'}

In [58]:
## could get weatherCode and use that as an ID
ID_ = hourlyConditions_mid.get('weatherCode'); ID_

'113'

In [59]:
#####

In [109]:
###writing out reduced dictionaries

with open('resort_reponses.json', 'w') as f:
    json.dump(resort_responses, f)